<a href="https://colab.research.google.com/github/galitneu/auto-eda-tool/blob/main/9330.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# --- שלב 0: ייבוא ספריות וחיבור לגוגל דרייב ---
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import RandomizedSearchCV

# חיבור סביבת העבודה לגוגל דרייב
from google.colab import drive
drive.mount('/content/drive')

# --- שלב 1: הגדרת נתיבים, טעינת הנתונים וסינון לפי שנה ---
DRIVE_PATH = '/content/drive/MyDrive/KaggleProject/'

try:
    df_train_raw = pd.read_csv(f'{DRIVE_PATH}Train.csv', low_memory=False)
    df_valid_raw = pd.read_csv(f'{DRIVE_PATH}Valid.csv', low_memory=False)
except FileNotFoundError:
    print(f"שגיאה: ודא שהקבצים 'Train.csv' ו-'Valid.csv' נמצאים בתיקייה: {DRIVE_PATH}")
    exit()

# --- סינון הנתונים לשנים 2008 ואילך ---
print("מבצע סינון ראשוני לשמירת נתונים משנת 2008 ואילך...")
df_train_raw['saledate'] = pd.to_datetime(df_train_raw['saledate'])
df_valid_raw['saledate'] = pd.to_datetime(df_valid_raw['saledate'])
df_train_raw = df_train_raw[df_train_raw['saledate'].dt.year >= 2008].copy()
df_valid_raw = df_valid_raw[df_valid_raw['saledate'].dt.year >= 2008].copy()
print(f"לאחר סינון, נותרו {len(df_train_raw)} רשומות בסט האימון.")


# --- שלב 2: איחוד קבצים לעיבוד אחיד ---

# !!! התיקון כאן: יצירת המשתנה train_labels *אחרי* הסינון !!!
train_labels = df_train_raw['SalePrice'].copy()
df_train_raw = df_train_raw.drop('SalePrice', axis=1)

df_train_raw['source'] = 'train'
df_valid_raw['source'] = 'valid'
df_combined = pd.concat([df_train_raw, df_valid_raw], ignore_index=True, sort=False)


# הסרת עמודות ID שאינן רלוונטיות למודל
columns_to_remove = ['MachineID']
existing_columns_to_remove = [col for col in columns_to_remove if col in df_combined.columns]
if existing_columns_to_remove:
    df_combined = df_combined.drop(existing_columns_to_remove, axis=1)
    print(f"הוסרו עמודות: {existing_columns_to_remove}")


# --- שלב 3: הנדסת מאפיינים ---
df_combined['saleYear'] = df_combined['saledate'].dt.year
df_combined = df_combined.drop('saledate', axis=1)
df_combined['machineAge'] = df_combined['saleYear'] - df_combined['YearMade']
valid_age_median = df_combined[df_combined['YearMade'] != 1000]['machineAge'].median()
df_combined.loc[df_combined['YearMade'] == 1000, 'machineAge'] = valid_age_median
df_combined['fiProductClassDesc'] = df_combined['fiProductClassDesc'].fillna('')
keywords_to_extract = ['excavator', 'dozer', 'loader', 'crawler', 'wheel', 'track']
for keyword in keywords_to_extract:
    df_combined[f'is_{keyword}'] = df_combined['fiProductClassDesc'].str.contains(keyword, case=False).astype(int)

# --- שלב 4: טיפול בערכים חסרים ---
numeric_cols_missing = ['MachineHoursCurrentMeter', 'auctioneerID']
for col in numeric_cols_missing:
    df_combined[col + '_is_missing'] = df_combined[col].isnull()
    median_val = df_combined[col].median()
    df_combined[col] = df_combined[col].fillna(median_val)
categorical_cols_missing = [col for col in df_combined.columns if pd.api.types.is_object_dtype(df_combined[col]) and df_combined[col].isnull().sum() > 0]
for col in categorical_cols_missing:
    df_combined[col] = df_combined[col].fillna('missing')

# --- שלב 5: המרת עמודות קטגוריאליות למספרים ---
source_col = df_combined['source']
df_combined = df_combined.drop('source', axis=1)
cols_to_drop = []
for col_name in df_combined.columns:
    if pd.api.types.is_object_dtype(df_combined[col_name]):
        num_unique_values = df_combined[col_name].nunique()
        if num_unique_values <= 5:
            dummies = pd.get_dummies(df_combined[col_name], prefix=col_name)
            df_combined = pd.concat([df_combined, dummies], axis=1)
            cols_to_drop.append(col_name)
        else:
            df_combined[col_name] = pd.Categorical(df_combined[col_name]).codes
df_combined = df_combined.drop(columns=cols_to_drop)
df_combined['source'] = source_col

# --- שלב 6: פיצול, אימון מודל והערכה (בשיטת Time-Based) ---
df_train_processed = df_combined[df_combined['source'] == 'train'].drop('source', axis=1).copy()
df_valid_processed = df_combined[df_combined['source'] == 'valid'].drop('source', axis=1).copy()
df_train_processed['SalePrice'] = train_labels.values # .values ensures correct assignment without index alignment issues

# 6.1: פיצול מבוסס-זמן לצורך הערכה מקומית
val_year = 2011
train_time_split = df_train_processed[df_train_processed['saleYear'] <= val_year - 1]
val_time_split = df_train_processed[df_train_processed['saleYear'] == val_year]
X_train_time = train_time_split.drop('SalePrice', axis=1)
y_train_time = train_time_split['SalePrice']
X_val_time = val_time_split.drop('SalePrice', axis=1)
y_val_time = val_time_split['SalePrice']

# 6.2: אימון והערכה על הפיצול המקומי
print("\n--- שלב הערכה: אימון על נתוני עבר (2008-2010) ובדיקה על 2011 ---")
model_time_split = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators= 150, min_samples_split= 5, min_samples_leaf= 4, max_features= 0.5, max_depth= 30)
model_time_split.fit(X_train_time, y_train_time)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
val_preds = model_time_split.predict(X_val_time)
val_rmsle_score = rmsle(y_val_time, val_preds)
print(f"ערך הפיספוס (RMSLE) על סט האימות של 2011 הוא: {val_rmsle_score:.5f}")

# --- שלב 7: אימון מודל סופי על כל נתוני האימון ---
print("\n" + "="*50 + "\n--- שלב סופי: אימון מחדש על כל נתוני האימון (2008-2011) ---")
X_full_train = df_train_processed.drop('SalePrice', axis=1)
y_full_train = df_train_processed['SalePrice']
final_model = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators= 150, min_samples_split= 5, min_samples_leaf= 4, max_features= 0.5, max_depth= 30)
final_model.fit(X_full_train, y_full_train)
print("אימון המודל הסופי הושלם.")

# --- שלב 8: יצירת קובץ הגשה על נתוני 2012 ---
print("\n" + "="*50 + "\n--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---")
train_cols = set(X_full_train.columns)
valid_cols = set(df_valid_processed.columns)
missing_in_valid = list(train_cols - valid_cols)
for c in missing_in_valid:
    df_valid_processed[c] = 0
df_valid_processed = df_valid_processed[X_full_train.columns]
valid_predictions = final_model.predict(df_valid_processed)
df_submission = pd.DataFrame({'SalesID': df_valid_raw['SalesID'], 'SalePrice': valid_predictions})
submission_filename = f'{DRIVE_PATH}submission.csv'
df_submission.to_csv(submission_filename, index=False)
print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה ב-Google Drive!")
print(df_submission.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
מבצע סינון ראשוני לשמירת נתונים משנת 2008 ואילך...
לאחר סינון, נותרו 152203 רשומות בסט האימון.
הוסרו עמודות: ['MachineID']

--- שלב הערכה: אימון על נתוני עבר (2008-2010) ובדיקה על 2011 ---
ערך הפיספוס (RMSLE) על סט האימות של 2011 הוא: 0.24327

--- שלב סופי: אימון מחדש על כל נתוני האימון (2008-2011) ---
אימון המודל הסופי הושלם.

--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---

קובץ ההגשה '/content/drive/MyDrive/KaggleProject/submission.csv' נשמר בהצלחה ב-Google Drive!
   SalesID     SalePrice
0  1222837  51021.783373
1  1222839  69508.731384
2  1222841  32635.222862
3  1222843  16445.745976
4  1222845  40607.849021


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# 6.3: יצירה והרצה של החיפוש האקראי
print("\n--- מתחיל חיפוש אקראי של היפר-פרמטרים... (זה עשוי לקחת זמן) ---")

# ... (הגדרת param_dist ו-rs נשארת זהה)
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_features': [0.5, 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rs_base_model = RandomForestRegressor(n_jobs=-1, random_state=42)

random_search = RandomizedSearchCV(estimator=rs_base_model,
                                   param_distributions=param_dist,
                                   n_iter=20,
                                   cv=3,
                                   verbose=2,
                                   random_state=42,
                                   n_jobs=1)

# התיקון כאן: הרצת החיפוש על האובייקט הנכון
random_search.fit(X_train_time, y_train_time)


# 6.4: הצגת הפרמטרים הטובים ביותר שנמצאו
print("\nהפרמטרים הטובים ביותר שנמצאו בחיפוש האקראי:")
# שימוש באובייקט הנכון כדי לקבל את התוצאות
best_model_random = random_search.best_estimator_
print(random_search.best_params_)

# 6.5: הערכת המודל עם הפרמטרים הטובים ביותר
val_preds = best_model_random.predict(X_val_time)
val_rmsle_score = rmsle(y_val_time, val_preds)
print(f"\nערך הפיספוס (RMSLE) עם המודל המכוונן הוא: {val_rmsle_score:.5f}")

# --- שלב 7: אימון מודל סופי עם הפרמטרים הטובים ביותר ---
print("\n" + "="*50 + "\n--- שלב סופי: אימון מחדש על כל נתוני האימון עם הפרמטרים המיטביים ---")
X_full_train = df_train_processed.drop('SalePrice', axis=1)
y_full_train = df_train_processed['SalePrice']

# התיקון כאן: שימוש בפרמטרים מהאובייקט הנכון
final_model = RandomForestRegressor(n_jobs=-1,
                                    random_state=42,
                                    **random_search.best_params_)

final_model.fit(X_full_train, y_full_train)
print("אימון המודל הסופי והמכוונן הושלם.")


--- מתחיל חיפוש אקראי של היפר-פרמטרים... (זה עשוי לקחת זמן) ---
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   3.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   4.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   4.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.6s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   6.6s
[CV] END max_depth=10, max_features=l

*italicized text*{'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.5, 'max_depth': 30}


In [ ]:
# --- שלב 8: יצירת קובץ הגשה על נתוני 2012 ---
print("\n" + "="*50 + "\n--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---")
train_cols = set(X_full_train.columns)
valid_cols = set(df_valid_processed.columns)
missing_in_valid = list(train_cols - valid_cols)
for c in missing_in_valid:
    df_valid_processed[c] = 0
df_valid_processed = df_valid_processed[X_full_train.columns]
valid_predictions = final_model.predict(df_valid_processed)
df_submission = pd.DataFrame({'SalesID': df_valid_raw['SalesID'], 'SalePrice': valid_predictions})
submission_filename = f'{DRIVE_PATH}submission.csv'
df_submission.to_csv(submission_filename, index=False)
print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה ב-Google Drive!")
print(df_submission.head())


--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---

קובץ ההגשה '/content/drive/MyDrive/KaggleProject/submission.csv' נשמר בהצלחה ב-Google Drive!
   SalesID     SalePrice
0  1222837  45631.943917
1  1222839  71520.674511
2  1222841  34326.652104
3  1222843  16260.024516
4  1222845  40855.074722


In [ ]:
!pip install shap
import shap
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

In [ ]:
# --- שלב 8: ניתוח חשיבות מאפיינים ---
print("\n--- ניתוח חשיבות מאפיינים ---")

# Feature Importance רגיל
feature_importance = pd.DataFrame({
    'feature': X_train_time.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 מאפיינים חשובים:")
print(feature_importance.head(15))

# Permutation Importance
print("\nמחשב Permutation Importance...")
perm_importance = permutation_importance(
    final_model, X_val_time, y_val_time,
    scoring='neg_mean_squared_log_error',
    n_repeats=3, random_state=42, n_jobs=1  # מספר חזרות קטן יותר וללא מקבילות
)

perm_importance_df = pd.DataFrame({
    'feature': X_train_time.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("Top 15 מאפיינים לפי Permutation Importance:")
print(perm_importance_df.head(15))

# --- שלב 9: SHAP Analysis ---
print("\n--- ניתוח SHAP ---")

try:
    # דגימה קטנה לניתוח SHAP (כדי לחסוך זמן)
    sample_size = min(500, len(X_train_time))  # הקטנת הדגימה
    X_sample = X_train_time.sample(n=sample_size, random_state=42)

    print("יוצר SHAP explainer...")
    # יצירת מודל חדש ללא n_jobs כדי למנוע בעיות pickling
    rf_for_shap = RandomForestRegressor(
        n_estimators=getattr(final_model, 'n_estimators', 100),
        max_depth=getattr(final_model, 'max_depth', None),
        min_samples_split=getattr(final_model, 'min_samples_split', 2),
        min_samples_leaf=getattr(final_model, 'min_samples_leaf', 1),
        max_features=getattr(final_model, 'max_features', 'sqrt'),
        random_state=42,
        n_jobs=1  # ללא מקבילות ל-SHAP
    )
    rf_for_shap.fit(X_train_time, y_train_time)

    explainer = shap.TreeExplainer(rf_for_shap)
    print("מחשב SHAP values...")
    shap_values = explainer.shap_values(X_sample)

    # SHAP Summary Plot
    print("יוצר SHAP summary plot...")
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_sample, max_display=20, show=False)
    plt.title('SHAP Summary Plot - Top 20 Features')
    plt.tight_layout()
    plt.savefig(f'{DRIVE_PATH}shap_summary_plot.png', dpi=300, bbox_inches='tight')
    plt.show()

    # SHAP Feature Importance
    shap_importance = pd.DataFrame({
        'feature': X_sample.columns,
        'shap_importance': np.abs(shap_values).mean(0)
    }).sort_values('shap_importance', ascending=False)

    print("Top 15 מאפיינים לפי SHAP:")
    print(shap_importance.head(15))

except Exception as e:
    print(f"שגיאה בניתוח SHAP: {e}")
    print("ממשיך בלי SHAP analysis...")
    # יצירת shap_importance ריק כדי שהקוד ימשיך לעבוד
    shap_importance = pd.DataFrame({
        'feature': X_train_time.columns,
        'shap_importance': np.zeros(len(X_train_time.columns))
    })

# --- שלב 10: הסרת מאפיינים לא חשובים ---
print("\n--- מסנן מאפיינים לא חשובים ---")

# בחירת מאפיינים על בסיס מספר מקורות חשיבות
top_features_rf = set(feature_importance.head(50)['feature'])
top_features_perm = set(perm_importance_df.head(50)['feature'])

# אם SHAP עבד, נשתמש בו גם
if 'shap_importance' in locals() and len(shap_importance) > 0 and shap_importance['shap_importance'].sum() > 0:
    top_features_shap = set(shap_importance.head(50)['feature'])
    # מאפיינים שמופיעים בלפחות 2 מתוך 3 הרשימות
    important_features = (top_features_rf & top_features_perm) | \
                        (top_features_rf & top_features_shap) | \
                        (top_features_perm & top_features_shap)
    print(f"משתמש ב-3 שיטות לבחירת מאפיינים")
else:
    # אם SHAP לא עבד, נשתמש רק ב-2 השיטות
    important_features = top_features_rf & top_features_perm
    if len(important_features) < 20:  # אם יש מעט מאפיינים משותפים
        important_features = top_features_rf | top_features_perm
    print(f"משתמש ב-2 שיטות לבחירת מאפיינים (ללא SHAP)")

print(f"נבחרו {len(important_features)} מאפיינים חשובים מתוך {len(X_train_time.columns)}")

# אימון מחדש עם מאפיינים נבחרים
X_train_time_selected = X_train_time[list(important_features)]
X_val_time_selected = X_val_time[list(important_features)]

final_rf_selected = RandomForestRegressor(
    n_estimators=getattr(final_model, 'n_estimators', 100),
    max_depth=getattr(final_model, 'max_depth', None),
    min_samples_split=getattr(final_model, 'min_samples_split', 2),
    min_samples_leaf=getattr(final_model, 'min_samples_leaf', 1),
    max_features=getattr(final_model, 'max_features', 'sqrt'),
    random_state=42,
    n_jobs=-1
)
final_rf_selected.fit(X_train_time_selected, y_train_time)

val_preds_selected = final_rf_selected.predict(X_val_time_selected)
val_rmsle_selected = rmsle(y_val_time, val_preds_selected)
print(f"RMSLE לאחר בחירת מאפיינים: {val_rmsle_selected:.5f}")

# --- שלב 11: אימון מודל סופי ---
print("\n" + "="*50 + "\n--- אימון מודל סופי ---")

X_full_train = df_train_processed.drop('SalePrice', axis=1)
y_full_train = df_train_processed['SalePrice']

# בחירת המודל הטוב יותר
if val_rmsle_selected < val_rmsle_current:
    print(f"משתמש במודל עם מאפיינים נבחרים (RMSLE: {val_rmsle_selected:.5f})")
    X_full_train_final = X_full_train[list(important_features)]
    best_model = RandomForestRegressor(
        n_estimators=getattr(final_model, 'n_estimators', 100),
        max_depth=getattr(final_model, 'max_depth', None),
        min_samples_split=getattr(final_model, 'min_samples_split', 2),
        min_samples_leaf=getattr(final_model, 'min_samples_leaf', 1),
        max_features=getattr(final_model, 'max_features', 'sqrt'),
        random_state=42,
        n_jobs=-1
    )
    selected_features = list(important_features)
    best_rmsle = val_rmsle_selected
else:
    print(f"משתמש במודל עם כל המאפיינים (RMSLE: {val_rmsle_current:.5f})")
    X_full_train_final = X_full_train
    best_model = final_model
    selected_features = list(X_full_train.columns)
    best_rmsle = val_rmsle_current

best_model.fit(X_full_train_final, y_full_train)
print("אימון המודל הסופי הושלם.")

# --- שלב 12: יצירת קובץ הגשה ---
print("\n" + "="*50 + "\n--- יצירת קובץ הגשה ---")

# הכנת נתוני validation
train_cols = set(selected_features)
valid_cols = set(df_valid_processed.columns)
missing_in_valid = list(train_cols - valid_cols)

for c in missing_in_valid:
    df_valid_processed[c] = 0

df_valid_final = df_valid_processed[selected_features]
valid_predictions = best_model.predict(df_valid_final)

# יצירת קובץ הגשה
df_submission = pd.DataFrame({
    'SalesID': df_valid_raw['SalesID'],
    'SalePrice': valid_predictions
})

submission_filename = f'{DRIVE_PATH}submission_improved.csv'
df_submission.to_csv(submission_filename, index=False)
print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה!")
print(f"דוגמה מהקובץ:")
print(df_submission.head())

# --- שלב 13: סיכום וויזואליזציות נוספות ---
print("\n" + "="*50 + "\n--- סיכום ---")
print(f"RMSLE בסיסי: {val_rmsle_basic:.5f}")
print(f"RMSLE סופי: {best_rmsle:.5f}")
print(f"שיפור: {((val_rmsle_basic - best_rmsle) / val_rmsle_basic * 100):.2f}%")
print(f"מספר מאפיינים במודל הסופי: {len(selected_features)}")

# Plot comparison של חשיבויות
plt.figure(figsize=(15, 10))

# Subplot 1: Feature Importance
plt.subplot(2, 2, 1)
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')
plt.gca().invert_yaxis()

# Subplot 2: Permutation Importance
plt.subplot(2, 2, 2)
top_perm = perm_importance_df.head(15)
plt.barh(range(len(top_perm)), top_perm['importance_mean'])
plt.yticks(range(len(top_perm)), top_perm['feature'])
plt.xlabel('Permutation Importance')
plt.title('Permutation Feature Importance')
plt.gca().invert_yaxis()

# Subplot 3: SHAP Importance (אם זמין)
if 'shap_importance' in locals() and len(shap_importance) > 0 and shap_importance['shap_importance'].sum() > 0:
    plt.subplot(2, 2, 3)
    top_shap = shap_importance.head(15)
    plt.barh(range(len(top_shap)), top_shap['shap_importance'])
    plt.yticks(range(len(top_shap)), top_shap['feature'])
    plt.xlabel('SHAP Importance')
    plt.title('SHAP Feature Importance')
    plt.gca().invert_yaxis()

    # Subplot 4: Prediction vs Actual
    plt.subplot(2, 2, 4)
else:
    # אם אין SHAP, נשתמש בכל השטח לגרף חיזוי מול אמיתי
    plt.subplot(2, 1, 2)  # שינוי לפורמט 2x1

# גרף חיזוי מול אמיתי
if val_rmsle_selected < val_rmsle_current:
    preds_to_plot = val_preds_selected
else:
    preds_to_plot = val_preds_current

plt.scatter(y_val_time, preds_to_plot, alpha=0.5)
plt.plot([y_val_time.min(), y_val_time.max()], [y_val_time.min(), y_val_time.max()], 'r--', lw=2)
plt.xlabel('מחיר אמיתי')
plt.ylabel('מחיר חזוי')
plt.title(f'חזוי מול אמיתי (RMSLE: {best_rmsle:.5f})')

plt.tight_layout()
plt.savefig(f'{DRIVE_PATH}feature_importance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nהתמונות נשמרו ב-{DRIVE_PATH}")
print("שיפורים נוספים לשקול:")
print("1. Ensemble של מספר מודלים (XGBoost, LightGBM)")
print("2. Cross-validation מתקדם יותר")
print("3. Target encoding עבור קטגוריות עם cardinality גבוה")
print("4. הנדסת מאפיינים נוספת על בסיס domain knowledge")


--- ניתוח חשיבות מאפיינים ---
Top 15 מאפיינים חשובים:
                   feature  importance
12             ProductSize    0.178980
5                 YearMade    0.144784
17               Enclosure    0.099059
27              machineAge    0.062746
13      fiProductClassDesc    0.061466
7              fiModelDesc    0.057976
2                  ModelID    0.046791
8              fiBaseModel    0.046735
9          fiSecondaryDesc    0.033237
1                MachineID    0.031314
94  Coupler_System_missing    0.023957
11       fiModelDescriptor    0.019733
21               Tire_Size    0.015490
26                saleYear    0.014471
97  Grouser_Tracks_missing    0.013393

מחשב Permutation Importance...
Top 15 מאפיינים לפי Permutation Importance:
                     feature  importance_mean  importance_std
12               ProductSize         0.235143        0.001444
5                   YearMade         0.158028        0.000232
13        fiProductClassDesc         0.059297        0.0001